# **Homework 5 - Bayes and K-Nearest**
# KDD Tuesdays 12:30 - 2:45 
## Jake Brulato

### **Import the packages**

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors
import sklearn.metrics

### **Read the data**

In [3]:
dataset = pd.read_csv("AdultUCI.csv", sep=',')
dataset

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,small
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,small
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,small
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,small
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,NaN
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,NaN
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,NaN
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,NaN


### **Check for the columns with nulls**

In [4]:
num = dataset.isnull().any().sum()
print("Columns with nulls: " , num)
dataset.isnull().any()

Columns with nulls:  4


age               False
workclass          True
fnlwgt            False
education         False
education-num     False
marital-status    False
occupation         True
relationship      False
race              False
sex               False
capital-gain      False
capital-loss      False
hours-per-week    False
native-country     True
income             True
dtype: bool

### **Drop the Nulls**

In [5]:
clean_data = dataset.dropna()
clean_data.info()
print(clean_data.isnull().sum())
clean_data

<class 'pandas.core.frame.DataFrame'>
Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   fnlwgt          30162 non-null  int64 
 3   education       30162 non-null  object
 4   education-num   30162 non-null  int64 
 5   marital-status  30162 non-null  object
 6   occupation      30162 non-null  object
 7   relationship    30162 non-null  object
 8   race            30162 non-null  object
 9   sex             30162 non-null  object
 10  capital-gain    30162 non-null  int64 
 11  capital-loss    30162 non-null  int64 
 12  hours-per-week  30162 non-null  int64 
 13  native-country  30162 non-null  object
 14  income          30162 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
age               0
workclass         0
fnlwgt            0
education         0
education-num     

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,small
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,small
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,small
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,small
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,small
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,large
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,small
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,small


### **Find the columns with more than 32 categorical variables and drop them**

In [6]:
categorical_cols = clean_data.select_dtypes(include=['object']).columns
high_level_categories = [col for col in categorical_cols if clean_data[col].nunique() > 32]
data_reduced = clean_data.drop(columns=high_level_categories)
data_reduced.head(), high_level_categories

(   age         workclass  fnlwgt  education  education-num  \
 0   39         State-gov   77516  Bachelors             13   
 1   50  Self-emp-not-inc   83311  Bachelors             13   
 2   38           Private  215646    HS-grad              9   
 3   53           Private  234721       11th              7   
 4   28           Private  338409  Bachelors             13   
 
        marital-status         occupation   relationship   race     sex  \
 0       Never-married       Adm-clerical  Not-in-family  White    Male   
 1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
 2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
 3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
 4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   
 
    capital-gain  capital-loss  hours-per-week income  
 0          2174             0              40  small  
 1             0             0              13  small  


### **Create the Label Encoders**

In [7]:
label_encoders = {}
for column in data_reduced.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data_reduced[column] = le.fit_transform(data_reduced[column])
    label_encoders[column] = le

### **Split into train and test data**

In [8]:
X = data_reduced.drop('income', axis=1)
y = data_reduced['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week
0,39,5,77516,9,13,4,0,1,4,1,2174,0,40
1,50,4,83311,9,13,2,3,0,4,1,0,0,13
2,38,2,215646,11,9,0,5,1,4,1,0,0,40
3,53,2,234721,1,7,2,5,0,2,1,0,0,40
4,28,2,338409,9,13,2,9,5,2,0,0,0,40


### **Create Naive Bayes Classifier**

In [9]:
classifier = GaussianNB()
gnb = classifier.fit(X_train,y_train)
gnb.predictions=gnb.predict(X_test)
gnb_accuracy = accuracy_score(y_test, gnb.predictions)

### **Confusion Matrix for Naive Bayes**

In [11]:
sklearn.metrics.confusion_matrix(y_test,gnb.predictions)

array([[ 483, 1067],
       [ 220, 4263]])

### **Create K-Neighbors Classifier**

In [14]:
classifier = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn = classifier.fit(X_train,y_train)
knn.predictions = knn.predict(X_test)
k_accuracy = accuracy_score(y_test, knn.predictions)

### **Confusion Matrix for K-Nearest**

In [15]:
sklearn.metrics.confusion_matrix(y_test,knn.predictions)

array([[ 482, 1068],
       [ 332, 4151]])

### **K-Neighbors and Naive Bayes Accuracy**

In [16]:
gnb_accuracy, k_accuracy

(0.7866732968672302, 0.7679429802751533)